In [1]:
import os

In [2]:
%pwd

'd:\\2 Code\\002 Feb 2024\\Kn\\Text-Summarizer-Project-KN-\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\2 Code\\002 Feb 2024\\Kn\\Text-Summarizer-Project-KN-'

Step 1: Update config.yaml

Step 3: # Third step : Update the entity

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

Step 4: Update the configuration manager in src config 

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml, create_directories


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
    

-  Step 5 : Components
- Here we write the code to download and unzip the files

In [8]:

import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    # function to download the data
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL, # source of the data from where it will be dowloaded
                filename = self.config.local_data_file # location where it will be stored
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  
    # function to unzip the downloaded files
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


    

- components creation is done
- Step 6: Now we need to create the pipeline

In [12]:
try:
    config = ConfigurationManager() # this is the class from Step 4
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config) # from Step 5
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-04-15 16:15:21,683: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-04-15 16:15:21,697: INFO: common: yaml file: params.yaml loaded successfully]
[2024-04-15 16:15:21,702: INFO: common: created directory at: artifacts]
[2024-04-15 16:15:21,705: INFO: common: created directory at: artifacts/data_ingestion]


[2024-04-15 16:15:25,922: INFO: 2324307505: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: BBB0:D6D09:71511:8D7BD:661D00CE
Accept-Ranges: bytes
Date: Mon, 15 Apr 2024 10:45:24 GMT
Via: 1.1 varnish
X-Served-By: cache-maa10244-MAA
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1713177924.199706,VS0,VE284
Vary: Authorization,Accept-Encoding,Origin
Access-Control-Allow-Origin: *
Cross-Origin-Resource-Policy: cross-origin
X-Fastly-Request-ID: aa026447af02b2e092e46e8fbd7f55140019ff7d
Expires: Mon, 15 Apr 2024 10:50:24 GMT
Source-Age: 0

]
